##### Create Delta Tables

In [2]:
countryCode = 'ch'

In [3]:
from pyspark.sql import functions as f

dfSSOL = spark.table('data_user_tim.operators_ssol_' + countryCode ).\
withColumn('addressOhub',\
           f.concat(\
                    f.when(f.col('street').isNotNull(),\
                           f.col('street')).\
                    otherwise(f.lit('')),\
                    f.lit(' '),\
                    f.when(f.col('housenumber').isNotNull(),\
                           f.col('housenumber')).\
                    otherwise(f.lit('')),
                    f.when(f.col('houseNumberExtension').isNotNull(),\
                           f.col('houseNumberExtension')).\
                    otherwise(f.lit('')))
          ).\
select('countryCode', 'operatorOhubId', 'operatorConcatID', f.col('name').alias('nameOhub'), 'addressOhub',
       f.col('zipCode').alias('zipCodeOhub'), f.col('city').alias('cityOhub'), f.col('channel').alias('channelOhub'))

dfSSOL.show()

+-----------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-------------------+
countryCode| operatorOhubId| operatorConcatID| nameOhub| addressOhub| zipCodeOhub| cityOhub| channelOhub|
+-----------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-------------------+
 CH|bc117017-9d7f-4fd...|CH~FUZZIT~672A215...| Rest. Linde Gmünder| Dorfstrasse 1 | 9054| Haslen Ai| null|
 CH|e5f527b2-b262-3fa...|CH~FUZZIT~20EC40A...|KBUV Kantonal Ber...|Ranflühschachen 528 | 3439| Ranflüh| null|
 CH|ef0fd9a1-69c0-3d9...|CH~FUZZIT~B71A947...| Restaurant Hofer| Hauptstrasse 23 | 4232| Fehren| null|
 CH|4062c0f2-1e93-41e...|CH~FUZZIT~189CBD6...|Seeblick GmbH Res...| Theaterstrasse 17 | 8400| Winterthur| null|
 CH|7ee9bb24-fea5-4ee...|CH~EMAKINA~412e9a...|TRACK4UFS-CH-FR T...|TRACK4UFS-CH-FR T...|TRACK4UFS-CH-FR T...|Track4ufs-ch-fr T...| other_institutions|
 CH|49e0343b-ac16-3b5...|CH~FUZZIT~6447952...| The Burger Terrasse| Nord, route 3 | 1972| Anzère| null|
 CH|d7e5321e-a24c-3b5...|CH~FUZZIT~5CF7FC9...|Restaurant Primav...|Obere Hauptgasse 46 | 3600| Thun| null|
 CH|1d9e7907-c408-40f...|CH~FUZZIT~667A9B1...|Gasthof Rössli Ze...| Hauptstrasse 57 | 4495| Zeglingen| null|
 CH|9d9066aa-ccc2-312...|CH~FUZZIT~D368173...| Gourmetbox GmbH|Freiburgstrasse 453 | 3018| Bern| null|
 CH|63aed8b4-ed33-4f2...|CH~EMAKINA~CD7EE7...| Localnet-Arena| Hofgutweg 24| 3400| Burgdorf| unknown|
 CH|ae1631d8-77ab-320...|CH~FUZZIT~1AD238B...|Restaurant/Hotel ...| Badstrasse 2 | 3423| Ersigen| null|
 CH|66da7885-f473-475...|CH~FUZZIT~14D4FAB...|Café Preisig, Kon...| Plaz 74 | 7482| Bergün/bravuogn| null|
 CH|52d6d4c7-90c0-34f...|CH~FUZZIT~79C58CB...| Mähdrescherei| Loch 1 | 3416|Affoltern Im Emme...| null|
 CH|b267a3b2-77e1-4da...| CH~DEX~83365|Wohnschule Freien...| Rüedistrasse 1| 8427| Freienstein| Education|
 CH|5e65127b-45d8-3b8...|CH~FUZZIT~BAC4DE8...|Schöni Partyservi...| Schwalbenweg 8 | 3627| Heimberg| null|
 CH|45dbfeec-f328-41e...|CH~FUZZIT~C099A17...|Wohn- und Pflegez...| Sihlwaldstrasse 2| 8135| Langnau Am Albis| null|
 CH|8f331a6e-8235-335...|CH~FUZZIT~B7FCF15...| Gastrovogel Sàrl| Saulcy, route de 5 | 2855| Glovelier| null|
 CH|c4e6cf53-e58e-401...|CH~EMAKINA~70c7e7...| Gef| Reuentalstrasse 4| 2558| Aegerten|other_public-sector|
 CH|3a4626cd-10d2-35f...|CH~FUZZIT~BC46B7D...|RiiBistro Freizei...|Unterer Rheinweg ...| 4057| Basel| null|
 CH|55d42fec-9100-43b...|CH~EMAKINA~07BBCC...| Hotel Kaubad| Kaustrasse 183| 9050| Appenzell| unknown|
+-----------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-------------------+
only showing top 20 rows

In [4]:
# Location where to save the Delta Table in the DBFS
deltaTable = "/mnt/datamodel/dev/sources/ohub/cleaned/operators_ssol_" + countryCode

# Location where to put the table in the Databricks database menu
hiveTable = "dev_sources_ohub.cleaned_operators_ssol_" + countryCode

In [5]:
# Write the data to a Delta Table
dfSSOL.write \
  .format("delta") \
  .mode("overwrite") \
  .option("overwriteSchema", "true") \
  .save(deltaTable)


In [6]:
# Load the Delta Table into the Data interface within Databricks.
sqlQuery1 = "drop table if exists " + hiveTable
sqlQuery2 = "create table " + hiveTable + " using delta location " + "'" + deltaTable + "'"
spark.sql(sqlQuery1)
spark.sql(sqlQuery2)

Out[20]: DataFrame[]

In [7]:
# The data in a Delta Table can be reordered to make it faster to work with the data (you might have a lot of small files). To improve the speed of read queries, you can use OPTIMIZE to collapse small files into larger ones. 
sqlQuery3 = "optimize " + hiveTable
spark.sql(sqlQuery3)


Out[21]: DataFrame[path: string, metrics: struct<numFilesAdded:bigint,numFilesRemoved:bigint,filesAdded:struct<min:bigint,max:bigint,avg:double,totalFiles:bigint,totalSize:bigint>,filesRemoved:struct<min:bigint,max:bigint,avg:double,totalFiles:bigint,totalSize:bigint>,partitionsOptimized:bigint,zOrderStats:struct<strategyName:string,inputCubeFiles:struct<num:bigint,size:bigint>,inputOtherFiles:struct<num:bigint,size:bigint>,inputNumCubes:bigint,mergedFiles:struct<num:bigint,size:bigint>,mergedNumCubes:bigint>,numBatches:bigint>]